# ETL with Python - Manipulating Large Masses of Data

**PROBLEMA**: uma organização recebe um conjunto de dados de um relatório de forma constante e é esperado que eles sejam organizados em um painel que sintetize todos eles de uma maneira clara. Esses dados de assinaturas são gerados diariamente e representam os produtos gerados por cada sistema distribuído em cada país. Como consequência, eles não possuem a mesma padronização.

Ao final será entregue uma aplicação que faz a carga de arquivos XLSX em Bancos de Dados.

Este estudo mostra **maturidade do processo** e **a visibilidade que se tem sobre os dados**.

O que será aprendido:

* **Organização de um projeto de Tratamentos de Dados**
    * Organização de as pastas
    * Criação de ambiente de execução (venv)

* Etapas de dados
* Conectar nos Bancos de Dados
* Garantir a confiabilidade dos dados
* Garantir a rastreabilidade dos dados
* Desenhar esquemas (tldraw - extensão no VSCode)


**CONCEITOS PARA ENTREVISTA**

Para demonstrar MATURIDADE e CONHECIMENTO DO SENTIDO das pequenas partes.

Muitas vezes, em entrevistas técnicas é comum a pesso achegar com ocódigo pronto. Muitas vezes é interessante explicar os principais conceitos (porque usar uma determinada solução? Como ela funciona? Para uso de VENV, organização do projeto e camadas de dados). **Demonstrar que sabe fazer o simples bem feito**.

Então saber falar sobre o desenvolvimento de SOLUÇÕES e não somente de CÓDIGOS.

**Regras do Tratamento de dados**

* Prezar pela confiabilidade e rastreabilidade dos dados

**Camadas de Armazenamento**

* RAW e READY
* BRONZE, SILVER e GOLD

**Ambiente Virtual Execução/Desenvolvimento**

**ETL de Dados**

* cookiecutter - módulo de python



**UTM tag**

UTM (Urchin Tracking Module ou Módulo de Rastreamento Urchin) é uma extensão de link de rastreamento que serve para entender a origem do tráfego.

Através da UTM, é possível saber onde foi publicado e qual o conteúdo e campanha veiculados a determinado link

## TAREFAS

* Consolidar diversos arquivos de planilhas em uma única fonte
    * Extrair informações de strings
    * Rastreabildiade dos dados - indicar qual a origem dos dados

## ROTEIRO DE PROJETO

1. <font color=orange><b>Iniciar um Projeto do Zero</b></font>
    1. Abrir pasta
        * `C:/www/python`
    2. Nova pasta para o Projeto
        * `mkdir make-data-netflix`
    3. Abrir o VSCode na pasta
        * `code make-data-netflix`
2. <font color=orange><b>Documentação</b></font>
    1. Crição de Pasta
        * `mkdir docs`
        * `cd docs`
        * `cat hot-to.md`
3. <font color=orange><b>Ambiente de Execução</b></font>
    1. Criação do ambiente
        * `python -m venv .venv`
    2. Ativação do ambiente
        * `.venv/Scripts/activate`
    3. Instalação de dependências
        * `pip install pandas, openpyxl, xlswriter, requests, polars`
        * `xlswriter` é mais performático na hora de escrever e permite operar de maneira mais atômica.
        * `pip install -r requirements.txt`
        * `poetry`
    4. Git
        * `.gitignore`
            * adicionar a pasta `src/data/raw/*`
            * adicionar a pasta `src/data/ready/*`
3. <font color=orange><b>Preparação</b></font>
    1.

* [Repositório de dados](https://www.youtube.com/redirect?event=live_chat&redir_token=QUFFLUhqbjFNdkpmNGhMTzROX2lJWGhXMUVYOW0xZWFtUXxBQ3Jtc0ttbUphNDV1dklmUEE4VjlnTWpONTVGSV8zeHZ5UG9CVWlHRWlfM1lvc3B5OV9FXzVkT2lyVk9LYWZqX3d3OVJVRWhZUDNNVFEtVWpObWxpWXlockhNbDN5WmZWUFBSLXdYRkJIaUduZFJCTE9VUTZ0RQ&q=https%3A%2F%2Fgithub.com%2Fdigitalinnovationone%2Fnetflix-dataset%2Ftree%2Fmain%2Fraw)

## Estrutura de Projeto de Tratamento de Dados

* `src`
    * _Tudo dentro do source é o código-fonte ou arquivos de textos, imagens etc. Tudo que estiver fora da `src` ou é documentação ou é configuração_
    * `data`
        * `raw`: dados brutos sem qualquer alteração
        * `ready`: dados que já passaram por alguns script de tratamento do projeto. Então é onde os dados refinados são armazenados.
    * `main.py`: se for só um arquivo ou uma aplicação inicial que chama todas as outras é possível usar um único arquivo.
    * [`scripts`]: pasta opcional para quando hpa mais do que um único arquivo de execução da aplicação. Isso serve para separar a camada de dados da camada de códigos.


Uma estrutura de projeto de API não é igual a uma estrutura de projeto de dados

"Transformação de data raw em dados refinados de negócios"

In [ ]:
import pandas as pd
import os
import glob

# Caminho para ler o caminho
folder_path = os.path.join(os.getcwd(), 'src', 'data', 'raw')

# Lista de arquivos
glob_unix_pattern = os.path.join(folder_path, '*.xlsx')
excel_files = glob.glob( glob_unix_pattern )

if not excel_files:
    print('No file found in the specified folder')
    raise ValueError('No file found in the specified folder')
else:
    # Dataframe - data in memory
    dfs = []

    for excel_file in excel_files:
        try:
            # Leitura de XLSX
            df_temp = pd.read_excel(excel_file)

            # Principio da rastreabilidade
            # Nome do arquivo
            df_temp['filename'] = os.path.basename(excel_file)

            # Extração da localização da tabela
            df_temp['location'] = df_temp['filename'].str.split('_')[-1]
            df_temp['location'] = df_temp['location'].str.split('.')[0].str.lower()

            # Extração de info sobre campaign - UTM
            # df_temp = df_temp['utm_link'].str.split('utm_source=').str[-1]
            df_temp = df_temp['utm_link'].str.extract(r'utm_campaign=(.*)')  # no extract recebe um padrão de regex

            df_temp['campaign'] = df_temp[df_temp['utm_source'].notnull()]['utm_campaign']
            df_temp['utm_source'] = df_temp[df_temp['utm_source'].notnull()]['utm_source']

            # Conjunto de tabelas
            dfs.append(df_temp)

        except Exception as e:
            print(f'Erro ao ler o arquivo {excel_file}: {e}')


if len(dfs) != 0:
    # Concatena todas as tabelas contidas no dfs em um única tabela
    result = pd.concat( dfs, ignore_index=True)

    # Cria o caminho da nova tabela tratada
    output_file = os.path.join(os.getcwd(), 'src', 'data', 'ready', 'netflix_data_cleaned.xlsx')

    # Criação da instância com especificação de motor (opcional)
    writer = pd.ExcelWriter(output_file, engine='xlsxwriter')  # mudança do motor de escrita. XLSXWRITER pode ser mais performático

    # Persistência da tabela em um arquivo excel e fechamento do arquivo
    result.to_excel(writer, sheet_name='netflix_data_cleaned', index=False)
    writer.save()
    writer.close()
else:
    print('Nenhum dado para ser salvo')





